In [1]:
import pandas as pd
import time
import websocket
import json, pprint, talib 
import numpy
from binance.client import Client
from binance.enums import *
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import timedelta
from dateutil.relativedelta import relativedelta

from pytrends.request import TrendReq
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

cmc = CoinMarketCapAPI('a01ace6b-f43c-439e-9fca-dc7f73e2a648')
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

def get_all_tokens():
    #filter out leverage tokens, non usd tokens, 
    all_pairs = pd.DataFrame(client.get_ticker())
    relev = all_pairs[all_pairs.symbol.str.endswith('USDT')]
    non_lev_list = relev[ ~( (relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')) | (relev.symbol.str.contains('BEAR')) | (relev.symbol.str.contains('BULL')) ) ]
    #filter out stable coins
    non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]
    #remove BCHSV, PAX, STRAT, XZC
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHSVUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STRATUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'XZCUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'AUDUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'EURUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GBPUSDT'].index )
    #sorted = non_stable_rm.sort_values(by='priceChangePercent', ascending=False)
    sorted = non_stable_rm.sort_values(by='symbol', key=lambda x: x.str.split('USDT').str[0])

    token_tickers = sorted
    symbols=token_tickers['symbol'].str.slice(0,-4)
    symbols = symbols.replace('GXS', 'GXC')
    symbols = symbols.replace('IOTA', 'MIOTA')
    symbolslist = ",".join(symbols)

    cmcinfo = cmc.cryptocurrency_info(symbol=symbolslist)
    names_info = cmcinfo.data
    names = []
    for ind in names_info:
        names.append(names_info[ind]['name'].replace('[', '').replace(']', '').replace('(', '') .replace(')', ''))
        
    sorted.insert(1, "name", names)
    #sorted['names'] = names

    return sorted

#format (symbol, interval, 120 mins ago, now) or (symbol, interval, start date, end date) "1 Dec, 2017"
def getminutedata(symbol, interval, start, end):
    if(end == 'now'):
        frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                        interval,
                                                        start + ' min ago UTC'))
    else:
        frame = pd.DataFrame(client.get_historical_klines(symbol, 
                                                        interval, 
                                                        start, 
                                                        end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame

def get_trends_3(coin, start, end):
    pytrends = TrendReq(hl='en-GB')
    data = pytrends.get_historical_interest(coin, 
                                    year_start=start[3], 
                                    month_start=start[2], 
                                    day_start=start[2], 
                                    hour_start=0, 
                                    year_end=end[3], 
                                    month_end=end[2], 
                                    day_end=end[1], 
                                    hour_end=0, 
                                    cat=0, 
                                    geo='', 
                                    gprop='', 
                                    sleep=0)
    data = data.drop('isPartial', axis=1)
    return data

def plot_coins(coin, HYPE, CLOSE):
    plt.figure(coin[1][0])
    plt.plot(CLOSE, color="green")
    plt.xlabel("Date & Time")
    plt.ylabel("Value $")
    plt2=plt.twinx()
    plt2.plot(HYPE)
    plt2.plot(HYPE.diff(periods=1))
    plt2.plot(HYPE.diff(periods=2))
    plt2.set_ylabel("Hype %")
    plt2.legend(['Close price'])
    plt.legend(['Buy Indicator', 'Rolling indicator 5 + 20%', 'Rolling indicator 5', 'differential 1', 'differential 2'])
    plt.title(coin[1])
    plt.show(block=False)

KeyboardInterrupt: 

In [ ]:
def backtest(HYPE, CLOSE, minbuy, minsell):
    HYPE['diff'] = HYPE.diff(periods=1)
    HYPE['Buy'] = (HYPE['diff'] > minbuy)
    HYPE['Sell'] = (HYPE['diff'] < minsell)
    HYPE['Close'] = CLOSE
    HYPE['Upward'] = HYPE['Close'].diff(periods=2)
    
    
    #print(HYPE)
    nHype = HYPE.to_numpy()
    holding = False
    buyprice = 0
    sellprice = 0
    tradeBuy = []
    tradeSell = []
    for row in range(len(nHype)):
        if ( (holding == False) and (nHype[row][2] == True) and (nHype[row][5] >= 0) ):
            buyprice = nHype[row][4]
            holding = True

        if (holding == True and nHype[row][3] == True):
            sellprice = nHype[row][4]
            tradeBuy.append(buyprice)
            tradeSell.append(sellprice)
            holding = False

    tradeDiff = []
    tradePerc = []

    for x in range(len(tradeBuy)-1):
        tradeDiff.append(tradeSell[x] - tradeBuy[x])
        tradePerc.append(tradeSell[x]/tradeBuy[x])

    tradeReturn = 1
    for x in tradePerc:
        tradeReturn *= x

    #print('Trade sums = ' + str(tradeDiff) )
    #print('Trade p/l% = ' + str(tradePerc) )
    result = 'Hold Return = ' + str(HYPE['Close'][-1]/HYPE['Close'][0]) + '   Bot Return = ' + str(tradeReturn)
    return result

In [ ]:
############################## MAIN CODE ################################

coins_info_df = get_all_tokens()
coins_id_df = coins_info_df[['symbol', 'name']].copy()
coins_id_array = coins_id_df.to_numpy()
coins_id_list = []
for x in range(len(coins_id_array)-1):
    coins_id_list.append([coins_id_array[x][0], [coins_id_array[x][1]]])

#testcoins = [ ['LRCUSDT', ['Loopring']], ['FTMUSDT', ['Fantom']], ['MINAUSDT', ['Mina']], ['NEARUSDT', ['NEAR Protocol']], ['ZECUSDT', ['Zcash']], ['LTOUSDT', ['LTO network']], ['WNXMUSDT', ['Wrapped NXM']], ['DUSKUSDT', ['Dusk Network']], ['IOTXUSDT', ['IoTeX']], ['IDEXUSDT', ['IDEX']],  ['LRCUSDT', ['Loopring']]]

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_1080/3645948845.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]


In [ ]:
f = open('results.txt', 'w')

time = str((60*24*7*10)+3)
time = str(60*24*7)
start = [0,19,11,2021]
end   = [0,21,11,2021]
for coin in coins_id_list:
    CLOSE = getminutedata(coin[0], '1m', time, 'now')['Close']
    HYPE = get_trends_3(coin[1], start, end)
  
    #plot_coins(coin, HYPE, CLOSE)
    
    for x in [0, 5, 10, 15, 20]:
        print('Threshold = ' + str(x))
        f.write('Threshold = ' + str(x))
        result = backtest(HYPE, CLOSE, x, 0)
        print(result)
        f.write(result)
    
    # backtest(HYPE, CLOSE)
    # pd.set_option('display.max_rows', None, 'display.max_columns', None)
    # print(HYPE)


    
plt.show()

#create a buy / sell line when diff passes 15-20, try all indicators and different values, then try on 100 coins in the last 2 weeks? each method saves the total theoretical gain just read of coin data value at ponit of crossing


#show average trade times , how many of these would happen simulatenously, print out number of tradse, average p/l%, average holding time

Threshold = 0
Hold Return = nan   Bot Return = 0.8879507287039253
Threshold = 5


ValueError: Columns must be same length as key